In [1]:
import os
import sys
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import manifold
from sklearn import decomposition
from matplotlib.backends.backend_pdf import PdfPages
import importlib

sys.path.append('../')
import catrace
from catrace import dataio
import catrace.process_time_trace as ptt
import catrace.plot_trace as pltr
import catrace.pattern_correlation as pcr
import catrace.exp_collection as ecl
from catrace.frame_time import convert_sec_to_frame
from catrace.trace_dataframe import concatenate_planes


In [2]:

exp_list = [( '2021-02-05-DpOBEM-JH9','phe'),
            ('2021-03-18-DpOBEM-JH10','phe'),
            ('2021-03-19-DpOBEM-JH10','phe'),
            ('2021-04-02-DpOBEM-JH11','phe'),
            ('2021-04-03-DpOBEM-JH11','phe'),
            ('2021-05-01-DpOBEM-JH13','phe'),
            ('2021-05-22-DpOBEM-JH14','phe'),
            ('2021-07-15-DpOBEM-N2'  ,'naive'),
            ('2021-07-16-DpOBEM-N3' ,'naive'),
            ('2021-07-30-DpOBEM-JH17','arg'),
            ('2021-07-31-DpOBEM-JH17','arg'),
            ('2021-09-02-DpOBEM-JH18','arg'),
            ('2021-09-03-DpOBEM-JH18','arg'),
            ('2021-09-04-DpOBEM-JH18','arg'),
            ('2021-09-15-DpOBEM-JH20','phe'),
            ('2021-09-17-DpOBEM-JH20','phe'),
            ('2021-09-18-DpOBEM-JH20','phe'),
            ('2021-09-29-DpOBEM-JH21','arg'),
            ('2021-10-01-DpOBEM-JH21','arg'),
            ('2021-10-02-DpOBEM-JH21','arg'),
            ('2021-11-10-DpOBEM-JH23','phetrp'),
            ('2021-11-14-DpOBEM-JH23','phetrp'),
            ('2021-11-24-DpOBEM-JH24','phetrp'),
            ('2021-11-27-DpOBEM-JH24','phetrp'),
            ('2021-11-26-DpOBEM-JH24','phetrp'),
            ('2021-12-10-DpOBEM-N4','naive'),
            ('2021-12-31-DpOBEM-N7','naive')]
region_list = ['Dp', 'OB']
frame_rate = 30/4
exp_info = dict(num_trial=3)
odor_list = ['phe', 'trp', 'arg', 'tdca', 'tca', 'gca', 'acsf', 'spont']

In [12]:
importlib.reload(ecl)
importlib.reload(dataio)
data_root_dir = '/media/hubo/WD_BoHu/BCE/Ca_imaging/results/'
dfovf_dict = dict()
for region in region_list:
    dfovf_dict[region] = dict()
    for exp in exp_list:
        exp_name = exp[0]
        print(exp_name, region)
        dfovf_dict[region][exp_name] = ecl.load_dfovf(data_root_dir,
                                                  exp_name, region)

2021-02-05-DpOBEM-JH9 Dp
2021-03-18-DpOBEM-JH10 Dp
2021-03-19-DpOBEM-JH10 Dp
2021-04-02-DpOBEM-JH11 Dp
2021-04-03-DpOBEM-JH11 Dp
2021-05-01-DpOBEM-JH13 Dp
2021-05-22-DpOBEM-JH14 Dp
2021-07-15-DpOBEM-N2 Dp
2021-07-16-DpOBEM-N3 Dp
2021-07-30-DpOBEM-JH17 Dp
2021-07-31-DpOBEM-JH17 Dp
2021-09-02-DpOBEM-JH18 Dp
2021-09-03-DpOBEM-JH18 Dp
2021-09-04-DpOBEM-JH18 Dp
2021-09-15-DpOBEM-JH20 Dp
2021-09-17-DpOBEM-JH20 Dp
2021-09-18-DpOBEM-JH20 Dp
2021-09-29-DpOBEM-JH21 Dp
2021-10-01-DpOBEM-JH21 Dp
2021-10-02-DpOBEM-JH21 Dp
2021-11-10-DpOBEM-JH23 Dp
2021-11-14-DpOBEM-JH23 Dp
2021-11-24-DpOBEM-JH24 Dp
2021-11-27-DpOBEM-JH24 Dp
2021-11-26-DpOBEM-JH24 Dp
2021-12-10-DpOBEM-N4 Dp
2021-12-31-DpOBEM-N7 Dp
2021-02-05-DpOBEM-JH9 OB
2021-03-18-DpOBEM-JH10 OB
2021-03-19-DpOBEM-JH10 OB
2021-04-02-DpOBEM-JH11 OB
2021-04-03-DpOBEM-JH11 OB
2021-05-01-DpOBEM-JH13 OB
2021-05-22-DpOBEM-JH14 OB
2021-07-15-DpOBEM-N2 OB
2021-07-16-DpOBEM-N3 OB
2021-07-30-DpOBEM-JH17 OB
2021-07-31-DpOBEM-JH17 OB
2021-09-02-DpOBEM-JH18 OB


In [13]:
csplus_dict = dict()
for exp in exp_list:
    if exp[1] not in csplus_dict.keys():
        csplus_dict[exp[1]] = []
    csplus_dict[exp[1]].append(exp[0])
csplus_dict

{'phe': ['2021-02-05-DpOBEM-JH9',
  '2021-03-18-DpOBEM-JH10',
  '2021-03-19-DpOBEM-JH10',
  '2021-04-02-DpOBEM-JH11',
  '2021-04-03-DpOBEM-JH11',
  '2021-05-01-DpOBEM-JH13',
  '2021-05-22-DpOBEM-JH14',
  '2021-09-15-DpOBEM-JH20',
  '2021-09-17-DpOBEM-JH20',
  '2021-09-18-DpOBEM-JH20'],
 'naive': ['2021-07-15-DpOBEM-N2',
  '2021-07-16-DpOBEM-N3',
  '2021-12-10-DpOBEM-N4',
  '2021-12-31-DpOBEM-N7'],
 'arg': ['2021-07-30-DpOBEM-JH17',
  '2021-07-31-DpOBEM-JH17',
  '2021-09-02-DpOBEM-JH18',
  '2021-09-03-DpOBEM-JH18',
  '2021-09-04-DpOBEM-JH18',
  '2021-09-29-DpOBEM-JH21',
  '2021-10-01-DpOBEM-JH21',
  '2021-10-02-DpOBEM-JH21'],
 'phetrp': ['2021-11-10-DpOBEM-JH23',
  '2021-11-14-DpOBEM-JH23',
  '2021-11-24-DpOBEM-JH24',
  '2021-11-27-DpOBEM-JH24',
  '2021-11-26-DpOBEM-JH24']}

In [14]:
importlib.reload(ptt)
select_neuron_explist = ecl.get_data_dict_decorator(exp_list, region_list, dfovf_dict, ptt.select_neuron)
thresh = 5.5
dfovf_select_dict = select_neuron_explist(thresh)

2021-02-05-DpOBEM-JH9 Dp
2021-03-18-DpOBEM-JH10 Dp
2021-03-19-DpOBEM-JH10 Dp
2021-04-02-DpOBEM-JH11 Dp
2021-04-03-DpOBEM-JH11 Dp
2021-05-01-DpOBEM-JH13 Dp
2021-05-22-DpOBEM-JH14 Dp
2021-07-15-DpOBEM-N2 Dp
2021-07-16-DpOBEM-N3 Dp
2021-07-30-DpOBEM-JH17 Dp
2021-07-31-DpOBEM-JH17 Dp
2021-09-02-DpOBEM-JH18 Dp
2021-09-03-DpOBEM-JH18 Dp
2021-09-04-DpOBEM-JH18 Dp
2021-09-15-DpOBEM-JH20 Dp
2021-09-17-DpOBEM-JH20 Dp
2021-09-18-DpOBEM-JH20 Dp
2021-09-29-DpOBEM-JH21 Dp
2021-10-01-DpOBEM-JH21 Dp
2021-10-02-DpOBEM-JH21 Dp
2021-11-10-DpOBEM-JH23 Dp
2021-11-14-DpOBEM-JH23 Dp
2021-11-24-DpOBEM-JH24 Dp
2021-11-27-DpOBEM-JH24 Dp
2021-11-26-DpOBEM-JH24 Dp
2021-12-10-DpOBEM-N4 Dp
2021-12-31-DpOBEM-N7 Dp
2021-02-05-DpOBEM-JH9 OB
2021-03-18-DpOBEM-JH10 OB
2021-03-19-DpOBEM-JH10 OB
2021-04-02-DpOBEM-JH11 OB
2021-04-03-DpOBEM-JH11 OB
2021-05-01-DpOBEM-JH13 OB
2021-05-22-DpOBEM-JH14 OB
2021-07-15-DpOBEM-N2 OB
2021-07-16-DpOBEM-N3 OB
2021-07-30-DpOBEM-JH17 OB
2021-07-31-DpOBEM-JH17 OB
2021-09-02-DpOBEM-JH18 OB


In [15]:
dfovf_select_dict['Dp']['2021-02-05-DpOBEM-JH9']

time                           0         1         2         3         4    \
odor  trial plane neuron                                                     
phe   0     0     1      -0.087107 -0.081762  0.120060 -0.175427 -0.118987   
                  2       0.026763 -0.120955 -0.217178  0.113037  0.196108   
                  3      -0.207659 -0.092794 -0.109698  0.130574 -0.328046   
                  4      -0.072826 -0.144551 -0.238825 -0.442386 -0.412755   
                  6       0.300471 -0.109048  0.189066 -0.014164  0.029926   
...                            ...       ...       ...       ...       ...   
spont 2     3     522     0.731620  0.348323 -0.386867 -0.034797 -0.237687   
                  523     0.043471  0.394242  0.128604  0.073303  0.346802   
                  524    -0.393986  0.127642  0.180911  0.137626  0.040839   
                  525     0.007720 -0.307461  0.762588 -0.350695  0.156781   
                  526    -0.090851  0.066170 -0.218405  0.102652 -0.315863   

time                           5         6         7         8         9    \
odor  trial plane neuron                                                     
phe   0     0     1      -0.116887  0.090842 -0.017838 -0.071818  0.087089   
                  2       0.195177  0.072438 -0.218604 -0.193099 -0.066215   
                  3      -0.503845 -0.381784 -0.304898 -0.098164 -0.081483   
                  4      -0.397014 -0.312698  0.154490 -0.221702  0.155312   
                  6       0.016210  0.510365 -0.046312  0.065659 -0.147255   
...                            ...       ...       ...       ...       ...   
spont 2     3     522     0.423448  0.437545 -0.177909  0.091185  0.241435   
                  523    -0.183021  0.028672  0.323927 -0.538901 -0.262179   
                  524    -0.221808  0.170519 -0.157827 -0.056733  0.089626   
                  525    -0.419990  0.089180  0.090950 -0.039694  0.151960   
                  526     0.607464 -0.254123 -0.196394 -0.114182 -0.245222   

time                      ...       155       156       157       158  \
odor  trial plane neuron  ...                                           
phe   0     0     1       ...  0.021412  0.069710 -0.069582  0.225703   
                  2       ... -0.042388  0.001541  0.237001  0.068077   
                  3       ... -0.167331  0.015945 -0.262990 -0.068162   
                  4       ... -0.178115 -0.417799  0.082159 -0.349345   
                  6       ...  0.334821 -0.004910 -0.109477 -0.127421   
...                       ...       ...       ...       ...       ...   
spont 2     3     522     ... -0.144183  0.760528 -0.031406 -0.159172   
                  523     ...  0.365137 -0.137958 -0.132927  0.129174   
                  524     ...  0.163707  0.423968 -0.206759 -0.153752   
                  525     ...  0.426430  0.147366  0.332204  0.685497   
                  526     ...  0.321455  0.156811  0.091312 -0.108401   

time                           159       160       161       162       163  \
odor  trial plane neuron                                                     
phe   0     0     1      -0.042487  0.183304  0.019115  0.194523 -0.087813   
                  2       0.065962 -0.260102  0.140047  0.159781 -0.280105   
                  3      -0.044443 -0.347097  0.055884 -0.303389 -0.247966   
                  4      -0.286121 -0.271007 -0.198012 -0.262711  0.018029   
                  6      -0.216323  0.847757 -0.238203  0.474594 -0.240833   
...                            ...       ...       ...       ...       ...   
spont 2     3     522    -0.252855 -0.289971  0.199322 -0.304782 -0.321378   
                  523    -0.067584  0.295588 -0.264538 -0.043549 -0.135069   
                  524     0.158002 -0.142342  0.342668  0.178960  0.376579   
                  525    -0.077731 -0.005423  0.003126  0.043799 -0.139419   
                  526    -0.266124  0.041897  0.181545  0.048766 -0.027278   

time 

In [21]:
import dataset_param as dtpar
dfovf_df = ecl.concatenate_df_from_db(dtpar.exp_list, dtpar.region_list, 'dfovf_select', dtpar.db_dir)

2021-02-05-DpOBEM-JH9 Dp


FileNotFoundError: [Errno 2] No such file or directory: '/media/hubo/WD_BoHu/Ca_imaging/results/JH_analysis/data/dfovf_select/2021-02-05-DpOBEM-JH9_Dp.pkl'